# 論文 17：變分有損自編碼器
## Xi Chen, Diederik P. Kingma 等人 (2016)

### VAE：具有學習潛在空間的生成模型

結合深度學習與變分推斷進行生成建模。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## 變分自編碼器（VAE）基礎

VAE 學習：
- **編碼器**：q(z|x) - 近似後驗
- **解碼器**：p(x|z) - 生成模型

**損失**：ELBO = 重建損失 + KL 散度

In [ ]:
def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

class VAE:
    def __init__(self, input_dim, hidden_dim, latent_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        
        # 編碼器：x -> h -> (mu, log_var)
        self.W_enc_h = np.random.randn(input_dim, hidden_dim) * 0.1
        self.b_enc_h = np.zeros(hidden_dim)
        
        self.W_mu = np.random.randn(hidden_dim, latent_dim) * 0.1
        self.b_mu = np.zeros(latent_dim)
        
        self.W_logvar = np.random.randn(hidden_dim, latent_dim) * 0.1
        self.b_logvar = np.zeros(latent_dim)
        
        # 解碼器：z -> h -> x_recon
        self.W_dec_h = np.random.randn(latent_dim, hidden_dim) * 0.1
        self.b_dec_h = np.zeros(hidden_dim)
        
        self.W_recon = np.random.randn(hidden_dim, input_dim) * 0.1
        self.b_recon = np.zeros(input_dim)
    
    def encode(self, x):
        """
        將輸入編碼為潛在分佈參數
        
        返回：q(z|x) 的 mu, log_var
        """
        h = relu(np.dot(x, self.W_enc_h) + self.b_enc_h)
        mu = np.dot(h, self.W_mu) + self.b_mu
        log_var = np.dot(h, self.W_logvar) + self.b_logvar
        return mu, log_var
    
    def reparameterize(self, mu, log_var):
        """
        重參數化技巧：z = mu + sigma * epsilon
        其中 epsilon ~ N(0, I)
        """
        std = np.exp(0.5 * log_var)
        epsilon = np.random.randn(*mu.shape)
        z = mu + std * epsilon
        return z
    
    def decode(self, z):
        """
        將潛在編碼解碼為重建
        
        返回：重建的 x
        """
        h = relu(np.dot(z, self.W_dec_h) + self.b_dec_h)
        x_recon = sigmoid(np.dot(h, self.W_recon) + self.b_recon)
        return x_recon
    
    def forward(self, x):
        """
        完整的前向傳遞
        """
        # 編碼
        mu, log_var = self.encode(x)
        
        # 採樣潛在變數
        z = self.reparameterize(mu, log_var)
        
        # 解碼
        x_recon = self.decode(z)
        
        return x_recon, mu, log_var, z
    
    def loss(self, x, x_recon, mu, log_var):
        """
        VAE 損失 = 重建損失 + KL 散度
        """
        # 重建損失（二元交叉熵）
        recon_loss = -np.sum(
            x * np.log(x_recon + 1e-8) + 
            (1 - x) * np.log(1 - x_recon + 1e-8)
        )
        
        # KL 散度：KL(q(z|x) || p(z))
        # 其中 p(z) = N(0, I)
        # KL = -0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        kl_loss = -0.5 * np.sum(1 + log_var - mu**2 - np.exp(log_var))
        
        return recon_loss + kl_loss, recon_loss, kl_loss

# 建立 VAE
input_dim = 16  # 例如展平的 4x4 圖像
hidden_dim = 32
latent_dim = 2  # 2D 用於視覺化

vae = VAE(input_dim, hidden_dim, latent_dim)
print(f"VAE 已建立：")
print(f"  輸入：{input_dim}")
print(f"  隱藏：{hidden_dim}")
print(f"  潛在：{latent_dim}")

## 生成合成資料

用於展示的簡單 4x4 模式

In [ ]:
def generate_patterns(num_samples=100):
    """
    生成簡單的 4x4 二元模式
    """
    data = []
    
    for i in range(num_samples):
        pattern = np.zeros((4, 4))
        
        if i % 4 == 0:
            # 水平線
            pattern[1:2, :] = 1
        elif i % 4 == 1:
            # 垂直線
            pattern[:, 2:3] = 1
        elif i % 4 == 2:
            # 對角線
            np.fill_diagonal(pattern, 1)
        else:
            # 角落正方形
            pattern[:2, :2] = 1
        
        # 添加小噪聲
        noise = np.random.randn(4, 4) * 0.05
        pattern = np.clip(pattern + noise, 0, 1)
        
        data.append(pattern.flatten())
    
    return np.array(data)

# 生成訓練資料
X_train = generate_patterns(200)

# 視覺化樣本
fig, axes = plt.subplots(1, 4, figsize=(12, 3))
for i, ax in enumerate(axes):
    ax.imshow(X_train[i].reshape(4, 4), cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'模式 {i}')
    ax.axis('off')
plt.suptitle('訓練資料樣本')
plt.show()

print(f"生成了 {len(X_train)} 個訓練樣本")

## 測試前向傳遞和損失

In [ ]:
# 在單一範例上測試
x = X_train[0:1]
x_recon, mu, log_var, z = vae.forward(x)

total_loss, recon_loss, kl_loss = vae.loss(x, x_recon, mu, log_var)

print(f"前向傳遞：")
print(f"  輸入形狀：{x.shape}")
print(f"  潛在 mu：{mu}")
print(f"  潛在 log_var：{log_var}")
print(f"  潛在 z：{z}")
print(f"  重建形狀：{x_recon.shape}")
print(f"\n損失：")
print(f"  總計：{total_loss:.4f}")
print(f"  重建：{recon_loss:.4f}")
print(f"  KL 散度：{kl_loss:.4f}")

# 視覺化重建
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
ax1.imshow(x.reshape(4, 4), cmap='gray', vmin=0, vmax=1)
ax1.set_title('原始')
ax1.axis('off')

ax2.imshow(x_recon.reshape(4, 4), cmap='gray', vmin=0, vmax=1)
ax2.set_title('重建（未訓練）')
ax2.axis('off')

plt.show()

## 視覺化潛在空間

由於 latent_dim=2，我們可以視覺化學習到的表示

In [ ]:
# 編碼所有訓練資料
latent_codes = []
pattern_types = []

for i, x in enumerate(X_train):
    mu, log_var = vae.encode(x.reshape(1, -1))
    latent_codes.append(mu[0])
    pattern_types.append(i % 4)

latent_codes = np.array(latent_codes)
pattern_types = np.array(pattern_types)

# 繪製潛在空間
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    latent_codes[:, 0], 
    latent_codes[:, 1], 
    c=pattern_types, 
    cmap='tab10', 
    alpha=0.6,
    s=50
)
plt.colorbar(scatter, label='模式類型')
plt.xlabel('潛在維度 1')
plt.ylabel('潛在維度 2')
plt.title('潛在空間（未訓練的 VAE）')
plt.grid(True, alpha=0.3)
plt.show()

print(f"潛在空間視覺化顯示編碼模式的分佈")

## 從先驗採樣並生成

採樣 z ~ N(0, I) 並解碼以生成新樣本

In [ ]:
# 從標準正態先驗採樣
num_samples = 8
z_samples = np.random.randn(num_samples, latent_dim)

# 生成樣本
generated = []
for z in z_samples:
    x_gen = vae.decode(z.reshape(1, -1))
    generated.append(x_gen[0])

# 視覺化生成的樣本
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes = axes.flatten()

for i, ax in enumerate(axes):
    ax.imshow(generated[i].reshape(4, 4), cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'z={z_samples[i][:2]}')
    ax.axis('off')

plt.suptitle('從先驗 p(z) = N(0, I) 生成的樣本', fontsize=14)
plt.tight_layout()
plt.show()

## 潛在空間中的插值

在潛在空間中兩點之間平滑插值

In [ ]:
# 編碼兩個不同的模式
x1 = X_train[0:1]  # 模式類型 0
x2 = X_train[1:2]  # 模式類型 1

mu1, _ = vae.encode(x1)
mu2, _ = vae.encode(x2)

# 插值
num_steps = 8
interpolated = []

for alpha in np.linspace(0, 1, num_steps):
    z_interp = (1 - alpha) * mu1 + alpha * mu2
    x_interp = vae.decode(z_interp)
    interpolated.append(x_interp[0])

# 視覺化插值
fig, axes = plt.subplots(1, num_steps, figsize=(16, 2))

for i, ax in enumerate(axes):
    ax.imshow(interpolated[i].reshape(4, 4), cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'α={i/(num_steps-1):.2f}')
    ax.axis('off')

plt.suptitle('潛在空間插值', fontsize=14, y=1.1)
plt.tight_layout()
plt.show()

print("平滑過渡顯示潛在空間的連續性")

## 重參數化技巧視覺化

In [ ]:
# 顯示從同一分佈的多次採樣
x = X_train[0:1]
mu, log_var = vae.encode(x)

# 多次採樣
num_samples = 100
z_samples = []
for _ in range(num_samples):
    z = vae.reparameterize(mu, log_var)
    z_samples.append(z[0])

z_samples = np.array(z_samples)

# 繪製分佈
plt.figure(figsize=(10, 8))
plt.scatter(z_samples[:, 0], z_samples[:, 1], alpha=0.3, s=20)
plt.scatter(mu[0, 0], mu[0, 1], color='red', s=200, marker='*', label='μ', zorder=5)

# 繪製 2 個標準差的橢圓
std = np.exp(0.5 * log_var[0])
theta = np.linspace(0, 2*np.pi, 100)
ellipse_x = mu[0, 0] + 2 * std[0] * np.cos(theta)
ellipse_y = mu[0, 1] + 2 * std[1] * np.sin(theta)
plt.plot(ellipse_x, ellipse_y, 'r--', label='2σ 邊界', linewidth=2)

plt.xlabel('z₁')
plt.ylabel('z₂')
plt.title('重參數化技巧：z = μ + σ ⊙ ε，其中 ε ~ N(0,I)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

print(f"μ = {mu[0]}")
print(f"σ = {std}")
print(f"樣本均值：{z_samples.mean(axis=0)}")
print(f"樣本標準差：{z_samples.std(axis=0)}")

## 關鍵要點

### VAE 架構：
1. **編碼器**：q_φ(z|x) - 將輸入映射到潛在分佈
2. **重參數化**：z = μ + σ ⊙ ε（使反向傳播可行）
3. **解碼器**：p_θ(x|z) - 從潛在編碼生成輸出

### 損失函數（ELBO）：
```
L = E[log p(x|z)] - KL(q(z|x) || p(z))
  = 重建損失 - KL 散度
```

### KL 散度：
- 正則化潛在空間使其接近先驗 p(z) = N(0, I)
- 防止過擬合
- 確保潛在空間平滑

### 重參數化技巧：
- 使採樣可微分
- z = μ(x) + σ(x) ⊙ ε，其中 ε ~ N(0, I)
- 梯度通過 μ 和 σ 流動

### 特性：
- **生成式**：可以採樣新資料
- **連續潛在空間**：平滑的插值
- **機率式**：建模不確定性
- **解糾纏表示**：（使用 β-VAE 等）

### 應用：
- 圖像生成
- 降維
- 半監督學習
- 異常檢測
- 資料增強

### 變體：
- **β-VAE**：加權 KL 以實現解糾纏
- **條件 VAE**：條件生成
- **階層式 VAE**：多層潛在層
- **VQ-VAE**：離散潛在變數